In [382]:
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_box_markup as show_span_markup, show_ascii_markup as show_markup, show_dep_ascii_markup as show_dep_markup
from slovnet import Morph, Syntax
from razdel import sentenize, tokenize

from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram, tag, custom, type, in_
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

import pymorphy2

pymorphy = pymorphy2.MorphAnalyzer(lang='ru')

navec = Navec.load('./data/navec_news_v1_1B_250K_300d_100q.tar')

ner = NER.load('./data/slovnet_ner_news_v1.tar')
morph = Morph.load('./data/slovnet_morph_news_v1.tar', batch_size=4)
syntax = Syntax.load('./data/slovnet_syntax_news_v1.tar')

ner.navec(navec)
morph.navec(navec)
syntax.navec(navec)

print("load: ok")

load: ok


In [383]:
source_txt ="""Совладельцы сети магазинов фиксированных низких цен Fix Price Артем Хачатрян и Сергей Ломакин стали долларовыми миллиардерами после выхода компании на IPO (первичное публичное размещение акций). Об этом свидетельствуют подсчеты Forbes.

Журнал оценил состояние каждого из них в 3,4 миллиарда долларов. Bloomberg написал о 3,6 миллиарда.

Fix Price разместила свои бумаги на Лондонской бирже по цене 9,75 доллара за штуку. Ломакину и Хачатряну до размещения принадлежало по 354 067 500 акций (41,66%) компаний, сейчас — по 301 151 876 акций (35,43%). Таким образом, в ходе IPO каждый из них продал по 52,9 миллиона акций, и заработал по 516 миллионов долларов, подсчитал Forbes.

Ранее сеть Fix Price, бумаги которой с 10 марта 2021 года начнут торговаться на Лондонской и Московской биржах, сообщила, что привлечет в ходе IPO 2 миллиарда долларов.

«Интерфакс» писал, что это крупнейшее IPO российской компании с 2010 года, когда «Русал» в рамках первичного публичного размещения акций привлек 2,24 миллиарда долларов.

Первые магазины сети Fix Price были открыты в 2007 году. Сейчас у компании 4279 магазинов в России, Беларуси, Казахстане, Узбекистане, Кыргызстане, Грузии и Латвии. Вначале весь ассортимент в Fix Price продавался по 30 рублей за товар, сейчас товары там стоят не дороже 250 рублей.

Артем Хачатрян и Сергей Ломакин в 1998 году стали сооснователями сети дискаунтеров «Копейка» (сейчас эти магазины известны под брендом «Пятерочка»). В 2007 году они продали свою долю корпорации «Уралсиб» за 220 миллионов долларов. Также Хачатрян и Ломакин инвестировали в сети «Модис» и «ЦентрОбувь».

В мае 2020 года Forbes писал, что в России больше 100 долларовых миллиардеров.

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-383-d68eb32fd5e9>, line 15)

In [384]:
class Token:
    def __init__(self, token):
        self.token = token
        self.morph = None
        self.syntax = None
    
    def set_morph(self, morph):
        self.morph = morph
        
    def set_syntax(self, syntax):
        self.syntax = syntax
        
    def __repr__(self):
        return self.token

class Sentence:
    def __init__(self, sentence):
        self.sentence = sentence
        self.tokens = []
        self.syntax_tree = [] 

        self.tokenize()
        
    @property
    def words(self):
        return list(map(str, self.tokens))

    def tokenize(self):
        self.tokens = [Token(_.text) for _ in tokenize(self.sentence)]
        
    def morph(self):
        markup = morph(self.words)

        for i, token in enumerate(markup.tokens):
            self.tokens[i].set_morph(token)
            
    def syntax(self):
        markup = syntax(self.words)

        for i, token in enumerate(markup.tokens):
            self.tokens[i].set_syntax(token)

            source = int(token.head_id) - 1
            target = int(token.id) - 1

            if source > 0 and source != target:  # skip root, loops
                self.syntax_tree.append([source, target, token.rel])
                
    def show_syntax(self):
        if self.syntax_tree == []:
            self.syntax()

        show_dep_markup(self.words, self.syntax_tree)
        
    def __str__(self):
        return self.sentence
        
    def __repr__(self):
        s = ''
        for tok in self.tokens:
            s += '[{}] '.format(tok)
        return s


class Text:
    def __init__(self, text):
        self.text = text
        self.sentences = []
        
        self.sentenize()
        
    def sentenize(self):
        self.sentences = [Sentence(_.text) for _ in sentenize(self.text)]
        
    def morph_all_sentences(self):
        for sent in self.sentences:
            sent.morph()
            
    def syntax_all_sentences(self):
        for sent in self.sentences:
            sent.syntax()

    def __str__(self):
        return self.text
        
    def __repr__(self):
        return '\n\n Sentence: '.join([str(sent) for sent in self.sentences])

# Сегментация

In [385]:
text = Text(source_txt)

# Морфологический и синтаксический разбор 

In [386]:
text.morph_all_sentences()

In [387]:
text.syntax_all_sentences()

In [388]:
sentence = text.sentences[4]

In [389]:
sentence.show_syntax()

        ┌► Fix        nsubj
        │  Price      
┌─┌─┌─┌─└─ разместила 
│ │ │ │ ┌► свои       det
│ │ │ └►└─ бумаги     obj
│ │ │ ┌──► на         case
│ │ │ │ ┌► Лондонской amod
│ │ └►└─└─ бирже      obl
│ │     ┌► по         case
│ └──►┌─└─ цене       obl
│     │ ┌► 9,75       nummod
│   ┌─└►└─ доллара    nmod
│   │   ┌► за         case
│   └──►└─ штуку      nmod
└────────► .          punct


# Извлечение именованных сущностей

In [390]:
markup = ner(sentence.sentence)

In [391]:
show_span_markup(markup.text, markup.spans)

# Поиск словосочетаний

In [392]:
def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            display(facts)

In [393]:
def exact_gram(tag):
    def check(v):
        parse = pymorphy.parse(v)[0]
        if not parse:
            return False

        return parse.tag.POS == tag
    
    return custom(check)

INT = type('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
ABBR = gram('Abbr')
VERB = gram('VERB')
PREP = gram('PREP')

EQ_NOUN = exact_gram('NOUN')

NUMBER = rule(
    INT.optional(),
    in_('.,').optional(),
    INT
)

gnc = gnc_relation()

In [421]:
NounPhrase = fact('NounPhrase', ['adj', 'noun'])

NumberPhrase = fact('NumberPhrase', ['number', 'noun'])

Fact = fact('Fact', ['value'])

NounPhraseRule = rule(
    ADJF.optional().repeatable().interpretation(
        NounPhrase.adj.inflected()
    ).match(gnc),

    EQ_NOUN.repeatable().interpretation(
        NounPhrase.noun.inflected()
    ).match(gnc)
).interpretation(NounPhrase).interpretation(Fact.value)

NumberPhraseRule = rule(
    NUMBER.interpretation(
        NumberPhrase.number
    ),
    EQ_NOUN.interpretation(
        NumberPhrase.noun
    )
).interpretation(NumberPhrase).interpretation(Fact.value)

AllRules = rule(or_(
    NumberPhraseRule,
    NounPhraseRule
)).interpretation(Fact)

In [422]:
show_matches(AllRules, sentence.sentence)

Fix Price разместила свои бумаги на Лондонской бирже по цене 9,75 
                     ───────────    ────────────────    ──── ─────
доллара за штуку.
───────    ───── 


[Fact(
     value=NounPhrase(
         adj='свой',
         noun='бумага'
     )
 ),
 Fact(
     value=NounPhrase(
         adj='лондонская',
         noun='биржа'
     )
 ),
 Fact(
     value=NounPhrase(
         adj=None,
         noun='цена'
     )
 ),
 Fact(
     value=NumberPhrase(
         number='9,75',
         noun='доллара'
     )
 ),
 Fact(
     value=NounPhrase(
         adj=None,
         noun='штука'
     )
 )]